{
 "cells": [
  {
   "cell_type": "markdown",
   "id": "af35bbf3",
   "metadata": {},
   "source": [
    "## 문제1. 경험적으로 추천해보기\n",
    "\n",
    "### 수행단계\n",
    " - 데이터는 제공되는 파일인 prob-0101.csv를 사용한다.\n",
    " - 영화 개수, 배급사 수, 감독 수, 출연진 수, 장르 수를 파악한다.\n",
    " - 감독, 출연진 등 중복 조건들에 대해 파악한다.\n",
    " - 본인이 추천하고 싶은 기준을 설정한다.\n",
    " - 본인의 기준에 의해 영화 몇개를 추천하는 것이 적당한지 실험해 본다.\n",
    " - 다양한 접근 조회 방법을 실험한다. (감독, 출연진 중복, 장르별 등)\n",
    " - 개인별로 정한 기준이 서로 어떻게 다른지 확인한다."
   ]
  },
 ]
}

In [7]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": None,
   "id": "data-loading",
   "metadata": {},
   "outputs": [],
   "source": [
    "import pandas as pd\n",
    "import re\n",
    "from collections import Counter, defaultdict\n",
    "from datetime import datetime\n",
    "\n",
    "# 데이터 불러오기\n",
    "df = pd.read_csv('prob-0101.csv')\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "id": "data-analysis",
   "metadata": {},
   "outputs": [],
   "source": [
    "# 데이터 분석\n",
    "print(\"\\n--- 데이터 분석 ---\")\n",
    "print(f\"전체 영화 개수: {len(df)}\")\n",
    "print(f\"중복 제외 배급사 수: {df['배급사'].nunique()}\")\n",
    "print(f\"중복 제외 감독 수: {df['감독'].nunique()}\")\n",
    "\n",
    "# 출연진 중복 제거\n",
    "unique_cast = set(','.join(df['출연진'].dropna()).split(','))\n",
    "print(f\"중복 제외 출연진 수: {len(unique_cast)}\")\n",
    "\n",
    "# 장르 중복 제거\n",
    "unique_genres = set(re.split(r'[·,/]', ','.join(df['장르'].dropna())))\n",
    "print(f\"중복 제외 장르 수: {len(unique_genres)}\")\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "id": "duplicate-check",
   "metadata": {},
   "outputs": [],
   "source": [
    "# 중복 조건 확인\n",
    "print(\"\\n--- 중복 조건 확인 ---\")\n",
    "# 같은 감독의 영화\n",
    "duplicate_directors = df.groupby('감독')['제목'].apply(list).loc[lambda x: x.map(len) > 1] # 감독별 영화 제목들을 리스트로 변환\n",
    "print(f\"같은 감독의 영화: {[f'{director}: {movies}' for director, movies in duplicate_directors.items()]}\")\n",
    "\n",
    "# 같은 출연진이 포함된 영화\n",
    "actor_to_movies = defaultdict(list)\n",
    "for _, row in df[['제목', '출연진']].dropna().iterrows():\n",
    "    for actor in row['출연진'].split(','):\n",
    "        actor_to_movies[actor.strip()].append(row['제목'])\n",
    "\n",
    "actors_in_multiple_movies = {actor: movies for actor, movies in actor_to_movies.items() if len(movies) > 1}\n",
    "print(\"\\n같은 출연진이 포함된 영화:\")\n",
    "if actors_in_multiple_movies:\n",
    "    for actor, movies in actors_in_multiple_movies.items():\n",
    "        print(f\"{actor}: {', '.join(movies)}\")\n",
    "else:\n",
    "    print(\"중복되는 출연진이 없습니다.\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "id": "season-recommendation",
   "metadata": {},
   "outputs": [],
   "source": [
    "# 계절에 맞는 영화 추천\n",
    "def get_season(date):\n",
    "    month = date.month\n",
    "    if month in [12, 1, 2]:\n",
    "        return '겨울'\n",
    "    elif month in [3, 4, 5]:\n",
    "        return '봄'\n",
    "    elif month in [6, 7, 8]:\n",
    "        return '여름'\n",
    "    else:\n",
    "        return '가을'\n",
    "\n",
    "df['개봉일'] = pd.to_datetime(df['개봉일'], errors='coerce')\n",
    "df['계절'] = df['개봉일'].apply(get_season)\n",
    "season_genre_map = {\n",
    "    '겨울': ['드라마'],\n",
    "    '봄': ['액션', '스릴러'],\n",
    "    '여름': ['공포', '스릴러'],\n",
    "    '가을': ['로맨스']\n",
    "}\n",
    "current_season = '겨울'  # 계절 입력 예시\n",
    "if current_season in season_genre_map:\n",
    "    season_genres = season_genre_map[current_season]\n",
    "    season_recommendations = df[df['장르'].apply(lambda x: any(genre in x for genre in season_genres))]\n",
    "    print(\"--- 계절 기반 영화 추천 ---\")\n",
    "    print(f\"{current_season} 계절에 추천하는 영화:\")\n",
    "    print(season_recommendations[['제목', '장르']])\n",
    "else:\n",
    "    print(f\"{current_season} 계절에 대한 추천이 없습니다.\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "id": "weighted-recommendation",
   "metadata": {},
   "outputs": [],
   "source": [
    "# 사용자 맞춤형 가중치 기반 추천\n",
    "user_weights = {'장르': 0.4, '출연진': 0.3, '개봉일': 0.3}\n",
    "df['사용자_추천점수'] = (\n",
    "    df['장르'].apply(lambda x: user_weights['장르'] if '드라마' in x else 0) +\n",
    "    ((df['출연진'].apply(lambda cast: sum(Counter(','.join(df['출연진'].dropna()).split(','))[actor.strip()] for actor in cast.split(',')) if pd.notnull(cast) else 0)) / df['출연진'].apply(lambda x: max(df['출연진'].apply(lambda y: len(y.split(',')) if pd.notnull(y) else 0))) * user_weights['출연진']) +\n",
    "    ((df['개봉일'] - df['개봉일'].min()).dt.days / (df['개봉일'].max() - df['개봉일'].min()).days * user_weights['개봉일'])\n",
    ")\n",
    "user_recommendations = df.sort_values(by='사용자_추천점수', ascending=False).head(5)\n",
    "print(\"--- 임의의 가중치 설정 기반 추천 ---\")\n",
    "print(\"사용자 맞춤형 추천 영화:\")\n",
    "print(user_recommendations[['제목', '사용자_추천점수']])"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "id": "date-analysis",
   "metadata": {},
   "outputs": [],
   "source": [
    "# 하루에 2편 이상 개봉한 날짜 출력\n",
    "release_count = df['개봉일'].value_counts()\n",
    "multiple_releases = release_count[release_count > 1]\n",
    "print(\"--- 하루에 2편 이상 개봉날짜 ---\")\n",
    "print(\"하루에 2편 이상 개봉한 날짜:\")\n",
    "print(multiple_releases.to_string(index=True))\n",
    "\n",
    "# 동시 개봉 영화 가장 많은 날짜\n",
    "most_releases_date = release_count.idxmax().date()\n",
    "most_releases_count = release_count.max()\n",
    "print(\"--- 동시 개봉 영화날짜 ---\")\n",
    "print(f\"동시 개봉 영화 가장 많은 날짜: {most_releases_date} ({most_releases_count}편)\")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.5"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}


{'cells': [{'cell_type': 'code',
   'execution_count': None,
   'id': 'data-loading',
   'metadata': {},
   'outputs': [],
   'source': ['import pandas as pd\n',
    'import re\n',
    'from collections import Counter, defaultdict\n',
    'from datetime import datetime\n',
    '\n',
    '# 데이터 불러오기\n',
    "df = pd.read_csv('prob-0101.csv')\n"]},
  {'cell_type': 'code',
   'execution_count': None,
   'id': 'data-analysis',
   'metadata': {},
   'outputs': [],
   'source': ['# 데이터 분석\n',
    'print("\\n--- 데이터 분석 ---")\n',
    'print(f"전체 영화 개수: {len(df)}")\n',
    'print(f"중복 제외 배급사 수: {df[\'배급사\'].nunique()}")\n',
    'print(f"중복 제외 감독 수: {df[\'감독\'].nunique()}")\n',
    '\n',
    '# 출연진 중복 제거\n',
    "unique_cast = set(','.join(df['출연진'].dropna()).split(','))\n",
    'print(f"중복 제외 출연진 수: {len(unique_cast)}")\n',
    '\n',
    '# 장르 중복 제거\n',
    "unique_genres = set(re.split(r'[·,/]', ','.join(df['장르'].dropna())))\n",
    'print(f"중복 제외 장르 수: {len(unique_genres)}")\n']},
  {'cell_ty